In [2]:
%pip install pyarrow torch sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings
warnings.filterwarnings("ignore")

import findspark
findspark.init()

from sentence_transformers import SentenceTransformer, util
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import pyarrow as pa
import pyarrow.parquet as pq

import pandas as pd
import numpy as np

In [4]:
parrallelism = 4
spark = SparkSession.builder \
                    .appName('Mpnet Sentence Embedding') \
                    .config("spark.dynamicAllocation.enabled", False) \
                    .config("spark.driver.memory", "4g") \
                    .config("spark.cores.max", parrallelism) \
                    .config("spark.executor.instances", parrallelism) \
                    .config("spark.executor.cores", 1) \
                    .config("spark.executor.memory", "8g") \
                    .enableHiveSupport() \
                    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [6]:
goods = spark.table("ruten.goods")
orders = spark.table("ruten.orders")
goods_vs_keywords = spark.table("ruten.goods_vs_keywords")

data = goods.join(orders, goods.item_id == orders.item_id, how="left") \
                .join(goods_vs_keywords, goods_vs_keywords.GNO == goods.item_id, how="left") \
                .where( col("orders.item_id").isNotNull() | col("goods_vs_keywords.gno").isNotNull() ) \
                .select(goods.item_id, goods.item_name) \
                .withColumn("item_name", lower(trim(regexp_replace(regexp_replace(col("item_name"), '&#\w+;', ''), '[^\u4e00-\u9fffa-zA-Z0-9]+', ' '))) ) \
                .distinct()

data.printSchema()

root
 |-- item_id: long (nullable = true)
 |-- item_name: string (nullable = true)



In [7]:
item_names_df = data.toPandas()
item_names_df

,item_id,item_name
0,10060814036060,二手音樂cd 98度單曲 征服未來
1,10060831298854,纖維彈簧透氣墊 沙發 汽車 電腦椅 輪椅 透氣椅墊 透氣坐墊 透氣座墊 小寵物墊 通風 散熱...
2,10060908825888,hikaru no go棋靈王便條紙
3,10060924029666,曹錦輝 tsao chin hui 2005 topps heritage 317
4,10061017636459,墨水王 評價9000 lxmark epson canon hp 高品質台灣填充墨水
...,...,...
1246576,22152635903312,jspb g5 標準版拉把
1246577,22152636113923,限fb買家陳先生下標購買rgeva貳號機
1246578,22152636835665,fun patch臂章圖鑑 限時預購
1246579,22152637425256,h 日版cd 安室奈美 break the rules song nation lovin it


In [9]:
embeddings = model.encode(item_names_df.item_name.values, show_progress_bar=True)

Batches: 100%|██████████| 313/313 [00:20<00:00, 15.43it/s]


In [10]:
item_names_df['embeddings'] = embeddings.tolist()
item_names_df.head()

,item_id,item_name,embeddings
0,10060814036060,二手音樂cd 98度單曲 征服未來,"[0.025065604597330093, 0.0536174438893795, -0...."
1,10060831298854,纖維彈簧透氣墊 沙發 汽車 電腦椅 輪椅 透氣椅墊 透氣坐墊 透氣座墊 小寵物墊 通風 散熱...,"[0.030799880623817444, -0.07103081047534943, -..."
2,10060908825888,hikaru no go棋靈王便條紙,"[0.064357228577137, 0.016072463244199753, -0.0..."
3,10060924029666,曹錦輝 tsao chin hui 2005 topps heritage 317,"[-0.08539468050003052, 0.13184863328933716, -0..."
4,10061017636459,墨水王 評價9000 lxmark epson canon hp 高品質台灣填充墨水,"[-0.028074568137526512, -0.014946662820875645,..."


In [11]:
item_names_df[['item_id', 'embeddings']].to_parquet("/tmp/mpnet_embeddings.parquet", engine='pyarrow', compression='snappy')

In [ ]:
!hdfs dfs -copyFromLocal /tmp/mpnet_embeddings.parquet "/ruten/mpnet_embeddings.parquet"